In [26]:
"""
Primary QB per team-season (2000–2024) using STARTERS from PBP:
- Starter for a game = QB with the team's first pass attempt in that game.
- Season "primary QB" = most game STARTS (tie → earliest first_week, then name).

Merges: birth_date, draft_year, draft_team from nfl_data_py.import_players.
Outputs per-season CSVs + one combined.
"""

import os
from pathlib import Path
from typing import List

import pandas as pd
import nfl_data_py as nfl

IN_CSV  = "games.csv"            # optional; used for sanity/week/season_type if present
OUT_DIR = "rosters"
SEASON_START, SEASON_END = 2000, 2024

# ---------------------------
# Helpers
# ---------------------------
def _ensure_dir(d: str | Path) -> None:
    Path(d).mkdir(parents=True, exist_ok=True)

def _canon_team(t: str) -> str:
    if not isinstance(t, str):
        return str(t)
    t = t.strip().upper()
    fix = {
        "OAK": "LV", "RAI": "LV",
        "SD": "LAC", "SDG": "LAC",
        "STL": "LAR", "LA": "LAR",
        "PHO": "ARI", "CRD": "ARI",
        "GNB": "GB", "NWE": "NE", "NOR": "NO", "SFO": "SF",
        "WSH": "WAS",
    }
    return fix.get(t, t)

def _load_players_small() -> tuple[pd.DataFrame, pd.DataFrame]:
    players = nfl.import_players()
    for c in ["full_name", "display_name", "player_name", "birth_date", "draft_year", "draft_team", "gsis_id"]:
        if c not in players.columns:
            players[c] = pd.NA
    players["full_name"] = (
        players["full_name"]
        .fillna(players["display_name"])
        .fillna(players["player_name"])
    )
    keep = ["gsis_id", "full_name", "birth_date", "draft_year", "draft_team"]
    players_small = players[keep].drop_duplicates("gsis_id").copy()
    names_small = players[["full_name", "birth_date", "draft_year", "draft_team"]].dropna(subset=["full_name"]).drop_duplicates().copy()
    return players_small, names_small

# ---------------------------
# Starter logic via PBP
# ---------------------------
def _derive_game_starters_from_pbp(season_start: int, season_end: int) -> pd.DataFrame:
    """
    For each (game_id, posteam), pick the earliest play with a passer_player_id.
    That QB is the starter for that team in that game.
    Returns: [season, week, game_id, team, starter_id, starter_name]
    """
    seasons = list(range(season_start, season_end + 1))
    pbp = nfl.import_pbp_data(seasons)  # contains season, week, game_id, posteam, passer_player_id/name, play_id, etc.
    # Regular season only (season_type is REG in nflfastR/nfl_data_py)
    if "season_type" in pbp.columns:
        pbp = pbp[pbp["season_type"].astype(str).str.upper().eq("REG")].copy()

    # Keep only plays with a credited passer
    need = ["season","week","game_id","posteam","passer_player_id","passer_player_name","play_id"]
    keep_cols = [c for c in need if c in pbp.columns]
    pbp2 = pbp[keep_cols].copy()
    pbp2 = pbp2[~pbp2["passer_player_id"].isna()].copy()

    # Earliest pass per (game_id, posteam)
    pbp2.sort_values(["game_id", "posteam", "play_id"], inplace=True)
    first_pass = pbp2.groupby(["game_id", "posteam"], as_index=False).first()
    first_pass.rename(columns={
        "posteam": "team",
        "passer_player_id": "starter_id",
        "passer_player_name": "starter_name"
    }, inplace=True)

    # Canon teams & clean names
    first_pass["team"] = first_pass["team"].astype(str).map(_canon_team)
    first_pass["starter_name"] = first_pass["starter_name"].astype(str).str.strip()

    # Some very rare games might not have a pass attempt for a team (weather/oddity).
    # Those teams simply won't appear here; downstream aggregation will only count games we can assign.
    return first_pass[["season","week","game_id","team","starter_id","starter_name"]]

def _pick_primary_qb_from_starts(game_starters: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate game-level starters to team-season primary QB by:
    - games_started = count of distinct game_id
    - tiebreakers: earliest first_week, then alphabetical name
    """
    agg = (
        game_starters.groupby(["season","team","starter_name","starter_id"], dropna=False)
        .agg(
            games_started=("game_id","nunique"),
            first_week=("week","min"),
            last_week=("week","max"),
        )
        .reset_index()
    )
    agg_sorted = agg.sort_values(
        ["season","team","games_started","first_week","starter_name"],
        ascending=[True, True, False, True, True]
    )
    primary = agg_sorted.drop_duplicates(subset=["season","team"], keep="first").copy()
    primary.rename(columns={"starter_name":"qb_name","starter_id":"qb_id"}, inplace=True)
    return primary.sort_values(["season","team"]).reset_index(drop=True)

def _merge_player_info(primary: pd.DataFrame,
                       players_small: pd.DataFrame,
                       names_small: pd.DataFrame) -> pd.DataFrame:
    out = primary.copy()
    out = out.rename(columns={"qb_name": "player_name", "qb_id": "player_id"})
    merged = out.merge(
        players_small.rename(columns={"gsis_id":"player_id","full_name":"players_full_name"}),
        on="player_id", how="left", validate="m:1"
    )
    need_fallback = merged["birth_date"].isna() & merged["draft_year"].isna() & merged["draft_team"].isna()
    if need_fallback.any():
        fb = merged.loc[need_fallback, ["season","team","player_name","player_id"]].merge(
            names_small.rename(columns={"full_name":"player_name"}),
            on="player_name", how="left", validate="m:1"
        )
        merged.loc[need_fallback, ["birth_date","draft_year","draft_team"]] = fb[["birth_date","draft_year","draft_team"]].values

    merged["team"] = merged["team"].map(_canon_team)
    cols = ["season","team","player_name","player_id","games_started","first_week","last_week","birth_date","draft_year","draft_team"]
    merged = merged[cols].sort_values(["season","team"]).reset_index(drop=True)
    return merged

# ---------------------------
# Main
# ---------------------------
def main(out_dir: str = OUT_DIR, season_start: int = SEASON_START, season_end: int = SEASON_END) -> None:
    _ensure_dir(out_dir)

    # 1) Build starters from PBP (starter of record per game)
    game_starters = _derive_game_starters_from_pbp(season_start, season_end)

    # 2) Collapse to one primary QB per team-season (by most GS)
    primary = _pick_primary_qb_from_starts(game_starters)

    # 3) Merge player metadata
    players_small, names_small = _load_players_small()
    merged = _merge_player_info(primary, players_small, names_small)

    # Debug: who lacks draft_team?
    missing_team = merged[merged["draft_team"].isna() | (merged["draft_team"] == "")]
    if not missing_team.empty:
        print("\n[NOTE] Players with missing draft_team:")
        print(missing_team[["season","team","player_name","draft_year"]].to_string(index=False))

    # 4) Save per-season + combined
    for season, chunk in merged.groupby("season"):
        out_path = Path(out_dir) / f"starters_qb_primary_{int(season)}.csv"
        chunk.sort_values(["team"]).to_csv(out_path, index=False)

    comb_path = Path(out_dir) / f"starters_qb_primary_{season_start}_{season_end}_combined.csv"
    merged.to_csv(comb_path, index=False)

    print(f"[OK] Wrote combined: {comb_path}  ({len(merged)} rows)")
    print(f"[OK] Wrote per-season files in: {Path(out_dir).resolve()}")
    print("\nSample:")
    print(merged.head(12).to_string(index=False))

if __name__ == "__main__":
    main()


2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.

[NOTE] Players with missing draft_team:
 season team  player_name  draft_year
   2000  CLE   D.Pederson         NaN
   2000  LAR     K.Warner         NaN
   2000  MIA    J.Fiedler         NaN
   2000  SEA      J.Kitna         NaN
   2000   SF     J.Garcia         NaN
   2001  CHI    J. Miller         NaN
   2001  CIN      J.Kitna         NaN
   2001  LAR     K.Warner         NaN
   2001  MIA    J.Fiedler         NaN
   2001   SF     J.Garcia         NaN
   2002  CHI     J.Miller         NaN
   2002  CIN      J.Kitna         NaN
   2002  DAL C.Hutchinson         NaN
   2002  MIA   J. Fiedler         NaN
   2002   SF     J.Garcia         NaN
   2002  WAS   S.Matthews         NaN
   2003  ATL    D.

In [27]:
import pandas as pd

# Path to the combined file you just created
combined_path = "rosters/starters_qb_primary_2000_2024_combined.csv"

# Load
df = pd.read_csv(combined_path)

# Find rows missing draft_team
missing_team = df[df["draft_team"].isna() | (df["draft_team"].astype(str).str.strip() == "")]

# Print nicely
if missing_team.empty:
    print("✅ No players with missing draft_team")
else:
    print("⚠️ Players with missing draft_team:\n")
    print(
        missing_team[["season", "team", "player_name", "draft_year"]]
        .sort_values(["season", "team"])
        .to_string(index=False)
    )

    # Optional: save to a CSV for review
    out_path = "rosters/missing_draft_team.csv"
    missing_team.to_csv(out_path, index=False)
    print(f"\nSaved list to {out_path}")


⚠️ Players with missing draft_team:

 season team  player_name  draft_year
   2000  CLE   D.Pederson         NaN
   2000  LAR     K.Warner         NaN
   2000  MIA    J.Fiedler         NaN
   2000  SEA      J.Kitna         NaN
   2000   SF     J.Garcia         NaN
   2001  CHI    J. Miller         NaN
   2001  CIN      J.Kitna         NaN
   2001  LAR     K.Warner         NaN
   2001  MIA    J.Fiedler         NaN
   2001   SF     J.Garcia         NaN
   2002  CHI     J.Miller         NaN
   2002  CIN      J.Kitna         NaN
   2002  DAL C.Hutchinson         NaN
   2002  MIA   J. Fiedler         NaN
   2002   SF     J.Garcia         NaN
   2002  WAS   S.Matthews         NaN
   2003  ATL    D.Johnson         NaN
   2003  CAR   J.Delhomme         NaN
   2003  CIN      J.Kitna         NaN
   2003  MIA    J.Fiedler         NaN
   2003   SF     J.Garcia         NaN
   2004  CAR   J.Delhomme         NaN
   2004  CLE     J.Garcia         NaN
   2004  NYG     K.Warner         NaN
   2005  ARI 

In [28]:
from pathlib import Path
import pandas as pd
import numpy as np

ROSTERS_COMB = Path("rosters/starters_qb_primary_2000_2024_combined.csv")
GAMES_CSV    = Path("games.csv")
OUT_DIR      = Path("rosters")
OUT_DIR.mkdir(exist_ok=True, parents=True)

# ---------- helpers ----------
_CANON = {
    "OAK":"LV","RAI":"LV",
    "SD":"LAC","SDG":"LAC",
    "STL":"LAR","LA":"LAR",
    "PHO":"ARI","CRD":"ARI",
    "GNB":"GB","NWE":"NE","NOR":"NO","SFO":"SF",
    "WSH":"WAS",
}
def canon(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().upper()
    return _CANON.get(s, s)

def to_date(x):
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT

def season_anchor(season:int):
    # Fallback anchor ~ start of season
    return pd.Timestamp(season, 9, 1)

def age_years(birth, date):
    if pd.isna(birth) or pd.isna(date): return np.nan
    return (date - birth).days / 365.2425

# ---------- load ----------
reg = pd.read_csv(ROSTERS_COMB)
games = pd.read_csv(GAMES_CSV)

# normalize
reg["team"] = reg["team"].map(canon)
reg["draft_team"] = reg["draft_team"].map(canon)
reg["drafted_by_team"] = (reg["team"] == reg["draft_team"])
reg["birth_date"] = pd.to_datetime(reg["birth_date"], errors="coerce")
reg = reg[(reg["season"] >= 2000) & (reg["season"] <= 2024)].copy()

games["game_type"] = games["game_type"].astype(str).str.upper()
games["home_team"] = games["home_team"].map(canon)
games["away_team"] = games["away_team"].map(canon)
games["gameday"] = games["gameday"].apply(to_date) if "gameday" in games.columns else pd.NaT

# =========================================================
# % drafted by team — REG (primary), Playoffs, Super Bowl
# =========================================================

# REG (primary) by year
yearly_reg = (reg.groupby("season", as_index=False)["drafted_by_team"]
                .mean()
                .rename(columns={"drafted_by_team":"pct_drafted"}))
yearly_reg["pct_drafted"] *= 100
overall_reg = yearly_reg["pct_drafted"].mean()
yearly_reg.to_csv(OUT_DIR/"yearly_pct_drafted_REG.csv", index=False)

# Playoffs (unique QBs per season)
is_playoff = games["game_type"].str.contains("POST|DIV|WC|CONF|CHAMP|SB") & ~games["game_type"].str.contains("REG")
playoff = games[is_playoff].copy()
ph = playoff.rename(columns={"home_team":"team","home_qb_name":"player_name","home_qb_id":"player_id"})[["season","team","player_name","player_id"]]
pa = playoff.rename(columns={"away_team":"team","away_qb_name":"player_name","away_qb_id":"player_id"})[["season","team","player_name","player_id"]]
p_long = pd.concat([ph, pa], ignore_index=True)
p_unique = p_long.drop_duplicates(["season","team","player_name"])

# prefer id join, fallback to name
reg_id = reg.dropna(subset=["player_id"])
p1 = p_unique.merge(reg_id[["season","team","player_id","drafted_by_team"]].drop_duplicates(),
                    on=["season","team","player_id"], how="left")
mask_fb = p1["drafted_by_team"].isna()
if mask_fb.any():
    p1.loc[mask_fb, :] = p1.loc[mask_fb].drop(columns=["drafted_by_team"]).merge(
        reg[["season","team","player_name","drafted_by_team"]].drop_duplicates(),
        on=["season","team","player_name"], how="left"
    )

playoff_yearly = (p1.groupby("season", as_index=False)["drafted_by_team"]
                    .mean()
                    .rename(columns={"drafted_by_team":"pct_drafted"}))
playoff_yearly["pct_drafted"] *= 100
overall_playoff = playoff_yearly["pct_drafted"].mean()
playoff_yearly.to_csv(OUT_DIR/"yearly_pct_drafted_playoffs.csv", index=False)

# Super Bowl
is_sb = games["game_type"].str.contains("SB|SUPER", na=False)
sb = games[is_sb].copy()
sbh = sb.rename(columns={"home_team":"team","home_qb_name":"player_name","home_qb_id":"player_id"})[["season","team","player_name","player_id"]]
sba = sb.rename(columns={"away_team":"team","away_qb_name":"player_name","away_qb_id":"player_id"})[["season","team","player_name","player_id"]]
sb_long = pd.concat([sbh, sba], ignore_index=True)

sb1 = sb_long.merge(reg_id[["season","team","player_id","drafted_by_team"]].drop_duplicates(),
                    on=["season","team","player_id"], how="left")
mask_fb2 = sb1["drafted_by_team"].isna()
if mask_fb2.any():
    sb1.loc[mask_fb2, :] = sb1.loc[mask_fb2].drop(columns=["drafted_by_team"]).merge(
        reg[["season","team","player_name","drafted_by_team"]].drop_duplicates(),
        on=["season","team","player_name"], how="left"
    )

sb_yearly = (sb1.groupby("season", as_index=False)["drafted_by_team"]
               .mean()
               .rename(columns={"drafted_by_team":"pct_drafted"}))
sb_yearly["pct_drafted"] *= 100
overall_sb = sb_yearly["pct_drafted"].mean()
sb_yearly.to_csv(OUT_DIR/"yearly_pct_drafted_superbowl.csv", index=False)

# =========================================================
# First start age, first bench age, & development window
# =========================================================

# panel of primaries
prim = reg[["season","team","player_id","player_name","birth_date"]].copy()

# span per (team, player)
span = (prim.sort_values(["team","player_id","season"])
             .groupby(["team","player_id","player_name","birth_date"], as_index=False)["season"]
             .agg(first_season="min", last_season="max"))

# first_start_date = earliest gameday in the FIRST season with that team/QB
games_reg = games[games["game_type"].str.contains("REG")].copy()
games_reg["gameday"] = games_reg["gameday"].apply(to_date) if "gameday" in games_reg else pd.NaT

def first_start_date(team, season, pid):
    mh = (games_reg["home_team"].eq(team) & games_reg["home_qb_id"].eq(pid) & games_reg["season"].eq(season))
    ma = (games_reg["away_team"].eq(team) & games_reg["away_qb_id"].eq(pid) & games_reg["season"].eq(season))
    d = games_reg.loc[mh | ma, "gameday"].min()
    return d if pd.notna(d) else season_anchor(season)

firsts = []
for _, r in span.iterrows():
    firsts.append({
        "team": r["team"],
        "player_id": r["player_id"],
        "first_season": int(r["first_season"]),
        "first_start_date": first_start_date(r["team"], int(r["first_season"]), r["player_id"])
    })
firsts = pd.DataFrame(firsts)

# bench_season = first season AFTER last where team has a DIFFERENT primary QB
# Build team->season primary id map
team_year_primary = prim.set_index(["team","season"])["player_id"].to_dict()

def bench_season(team, last_season, pid):
    ny = int(last_season) + 1
    if (team, ny) in team_year_primary and team_year_primary[(team, ny)] != pid:
        return ny
    return np.nan

span["bench_season"] = [bench_season(r["team"], r["last_season"], r["player_id"]) for _, r in span.iterrows()]

# bench_date = first gameday for that team in bench_season (regardless of QB)
def first_team_game_date(team, season):
    mask = (games_reg["season"].eq(season) & (games_reg["home_team"].eq(team) | games_reg["away_team"].eq(team)))
    d = games_reg.loc[mask, "gameday"].min()
    return d if pd.notna(d) else season_anchor(int(season))

benches = []
for _, r in span.dropna(subset=["bench_season"]).iterrows():
    benches.append({
        "team": r["team"],
        "player_id": r["player_id"],
        "bench_season": int(r["bench_season"]),
        "bench_date": first_team_game_date(r["team"], int(r["bench_season"]))
    })
benches = pd.DataFrame(benches) if benches else pd.DataFrame(columns=["team","player_id","bench_season","bench_date"])

# merge dates
dev = span.merge(firsts, on=["team","player_id","first_season"], how="left")
if not benches.empty:
    dev = dev.merge(benches, on=["team","player_id","bench_season"], how="left")
else:
    dev["bench_date"] = pd.NaT

# ages and window
dev["first_start_age"] = [age_years(b, d) for b, d in zip(dev["birth_date"], dev["first_start_date"])]
dev["bench_age"]       = [age_years(b, d) for b, d in zip(dev["birth_date"], dev["bench_date"])]
dev["seasons_until_bench"] = dev["bench_season"] - dev["first_season"]

# cohort averages by first season
cohort = (dev.groupby("first_season", as_index=False)
            .agg(
                avg_first_start_age = ("first_start_age", "mean"),
                avg_bench_age       = ("bench_age", "mean"),
                avg_window_seasons  = ("seasons_until_bench", "mean"),
                n_qbs               = ("player_id", "nunique")
            )
            .rename(columns={"first_season":"season"}))

cohort.to_csv(OUT_DIR/"qb_dev_window_by_cohort.csv", index=False)

# --------- report ----------
print("\n=== Drafted-by-team shares (2000–2024, season averages) ===")
print(f"Regular season (primary QBs): {yearly_reg['pct_drafted'].mean():.1f}%")
print(f"Playoffs (unique QBs)       : {playoff_yearly['pct_drafted'].mean():.1f}%")
print(f"Super Bowl QBs              : {sb_yearly['pct_drafted'].mean():.1f}%")

print("\nSaved:")
print("  rosters/yearly_pct_drafted_REG.csv")
print("  rosters/yearly_pct_drafted_playoffs.csv")
print("  rosters/yearly_pct_drafted_superbowl.csv")
print("  rosters/qb_dev_window_by_cohort.csv")

print("\n=== Dev-window cohort preview ===")
print(cohort.to_string(index=False))



=== Drafted-by-team shares (2000–2024, season averages) ===
Regular season (primary QBs): 55.2%
Playoffs (unique QBs)       : 59.6%
Super Bowl QBs              : 68.0%

Saved:
  rosters/yearly_pct_drafted_REG.csv
  rosters/yearly_pct_drafted_playoffs.csv
  rosters/yearly_pct_drafted_superbowl.csv
  rosters/qb_dev_window_by_cohort.csv

=== Dev-window cohort preview ===
 season  avg_first_start_age  avg_bench_age  avg_window_seasons  n_qbs
   2000            28.679929      31.545069            2.900000     31
   2001            29.290312      33.002980            4.545455     15
   2002            28.463099      31.020715            2.750000     15
   2003            27.258146      30.688378            3.100000     12
   2004            29.906369      33.028846            3.153846     13
   2005            28.822943      32.416570            3.636364     11
   2006            26.850865      31.267951            4.461538     13
   2007            29.140001      31.121104            2.000

In [32]:
from pathlib import Path
import pandas as pd
import numpy as np

# =========================
# Config & paths
# =========================
ROSTERS_COMB = Path("rosters/starters_qb_primary_2000_2024_combined.csv")
GAMES_CSV    = Path("games.csv")
OUT_DIR      = Path("rosters")
OUT_DIR.mkdir(exist_ok=True, parents=True)

SEASON_START, SEASON_END = 2000, 2024

# =========================
# Helpers
# =========================
_CANON = {
    "OAK":"LV","RAI":"LV",
    "SD":"LAC","SDG":"LAC",
    "STL":"LAR","LA":"LAR",
    "PHO":"ARI","CRD":"ARI",
    "GNB":"GB","NWE":"NE","NOR":"NO","SFO":"SF",
    "WSH":"WAS",
}
def canon(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().upper()
    return _CANON.get(s, s)

def to_date(x):
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT

def season_anchor(season:int):
    return pd.Timestamp(int(season), 9, 1)

def age_years(birth, date):
    if pd.isna(birth) or pd.isna(date): return np.nan
    return (date - birth).days / 365.2425

def same_franchise(series_team: pd.Series, series_draft_team: pd.Series) -> pd.Series:
    """Null-safe equality: returns boolean Series with no NA (missing → False)."""
    a = series_team.astype("string").fillna("_NA_")
    b = series_draft_team.astype("string").fillna("_NA_")
    return a == b

# =========================
# Load & normalize
# =========================
reg_all = pd.read_csv(ROSTERS_COMB)
games   = pd.read_csv(GAMES_CSV)

# Canonicalize first
reg_all["team"]       = reg_all["team"].map(canon)
reg_all["draft_team"] = reg_all["draft_team"].map(canon)

# Ensure expected columns exist
if "birth_date" not in reg_all.columns:
    reg_all["birth_date"] = pd.NaT  # create if missing
reg_all["birth_date"] = pd.to_datetime(reg_all["birth_date"], errors="coerce")

need = ["season","team","player_id","player_name","draft_team"]
miss = [c for c in need if c not in reg_all.columns]
if miss:
    raise KeyError(f"Missing columns in roster file: {miss}")

# Season window
reg_all = reg_all.query("@SEASON_START <= season <= @SEASON_END").copy()

# Guaranteed drafted flag (NULL-SAFE)
reg_all["drafted_by_team"] = same_franchise(reg_all["team"], reg_all["draft_team"])

# Drafted-only starters subset
reg_drafted = reg_all.loc[reg_all["drafted_by_team"]].copy()

# Normalize games
games["game_type"] = games["game_type"].astype(str).str.upper()
games["home_team"] = games["home_team"].map(canon)
games["away_team"] = games["away_team"].map(canon)
games["gameday"]   = games["gameday"].apply(to_date) if "gameday" in games.columns else pd.NaT

# =========================
# Regular-season % drafted-by-team
# =========================
yearly_reg = (reg_all.groupby("season", as_index=False)["drafted_by_team"]
                .mean()
                .rename(columns={"drafted_by_team":"pct_drafted"}))
yearly_reg["pct_drafted"] *= 100.0
overall_reg = yearly_reg["pct_drafted"].mean()

# Also: count of drafted-only starters per season
yearly_reg_counts = (reg_drafted.groupby("season")["team"]
                       .nunique()
                       .reset_index(name="n_drafted_starters"))

yearly_reg.to_csv(OUT_DIR/"yearly_pct_drafted_REG.csv", index=False)
yearly_reg_counts.to_csv(OUT_DIR/"yearly_count_drafted_REG.csv", index=False)

# =========================
# Playoffs % drafted-by-team (unique QBs per season)
# =========================
is_playoff = games["game_type"].str.contains("POST|DIV|WC|CONF|CHAMP|SB") & ~games["game_type"].str.contains("REG")
playoff = games.loc[is_playoff].copy()

ph = playoff.rename(columns={"home_team":"team","home_qb_name":"player_name","home_qb_id":"player_id"})[
    ["season","team","player_name","player_id"]]
pa = playoff.rename(columns={"away_team":"team","away_qb_name":"player_name","away_qb_id":"player_id"})[
    ["season","team","player_name","player_id"]]
p_long = pd.concat([ph, pa], ignore_index=True)

# Dedup (prefer id rows over name-only)
p_ids   = p_long.dropna(subset=["player_id"]).drop_duplicates(["season","team","player_id"])
p_names = p_long[p_long["player_id"].isna()].drop_duplicates(["season","team","player_name"])
p_unique = pd.concat([p_ids, p_names], ignore_index=True)

# ID-join first
reg_id = (reg_all.dropna(subset=["player_id"])
                 [["season","team","player_id","drafted_by_team","draft_team"]]
                 .drop_duplicates())
p1 = p_unique.merge(reg_id[["season","team","player_id","drafted_by_team"]],
                    on=["season","team","player_id"], how="left")

# Fallback by NAME: compute drafted flag fresh from draft_team (NULL-SAFE)
need_fb = p1["drafted_by_team"].isna()
if need_fb.any():
    fb = (p1.loc[need_fb, ["season","team","player_name"]]
            .merge(reg_all[["season","team","player_name","draft_team"]].drop_duplicates(),
                   on=["season","team","player_name"], how="left"))
    drafted_fb = same_franchise(fb["team"], fb["draft_team"]).to_numpy()
    p1.loc[need_fb, "drafted_by_team"] = drafted_fb

playoff_yearly = (p1.groupby("season", as_index=False)["drafted_by_team"]
                    .mean()
                    .rename(columns={"drafted_by_team":"pct_drafted"}))
playoff_yearly["pct_drafted"] *= 100.0
overall_playoff = playoff_yearly["pct_drafted"].mean()

# Optional dataset: drafted-only playoff QBs
playoff_only_drafted = p1.loc[p1["drafted_by_team"].fillna(False)].copy()

playoff_yearly.to_csv(OUT_DIR/"yearly_pct_drafted_playoffs.csv", index=False)
playoff_only_drafted.to_csv(OUT_DIR/"playoff_qbs_drafted_only.csv", index=False)

# =========================
# Super Bowl % drafted-by-team
# =========================
is_sb = games["game_type"].str.contains("SB|SUPER", na=False)
sb = games.loc[is_sb].copy()

sbh = sb.rename(columns={"home_team":"team","home_qb_name":"player_name","home_qb_id":"player_id"})[
    ["season","team","player_name","player_id"]]
sba = sb.rename(columns={"away_team":"team","away_qb_name":"player_name","away_qb_id":"player_id"})[
    ["season","team","player_name","player_id"]]
sb_long = pd.concat([sbh, sba], ignore_index=True)

sb_ids   = sb_long.dropna(subset=["player_id"]).drop_duplicates(["season","team","player_id"])
sb_names = sb_long[sb_long["player_id"].isna()].drop_duplicates(["season","team","player_name"])
sb_unique = pd.concat([sb_ids, sb_names], ignore_index=True)

sb1 = sb_unique.merge(reg_id[["season","team","player_id","drafted_by_team"]],
                      on=["season","team","player_id"], how="left")

need_fb2 = sb1["drafted_by_team"].isna()
if need_fb2.any():
    fb2 = (sb1.loc[need_fb2, ["season","team","player_name"]]
             .merge(reg_all[["season","team","player_name","draft_team"]].drop_duplicates(),
                    on=["season","team","player_name"], how="left"))
    drafted_fb2 = same_franchise(fb2["team"], fb2["draft_team"]).to_numpy()
    sb1.loc[need_fb2, "drafted_by_team"] = drafted_fb2

sb_yearly = (sb1.groupby("season", as_index=False)["drafted_by_team"]
               .mean()
               .rename(columns={"drafted_by_team":"pct_drafted"}))
sb_yearly["pct_drafted"] *= 100.0
overall_sb = sb_yearly["pct_drafted"].mean()

sb_only_drafted = sb1.loc[sb1["drafted_by_team"].fillna(False)].copy()

sb_yearly.to_csv(OUT_DIR/"yearly_pct_drafted_superbowl.csv", index=False)
sb_only_drafted.to_csv(OUT_DIR/"superbowl_qbs_drafted_only.csv", index=False)

# =========================
# Development window (drafted-only starters)
# =========================
prim = reg_drafted[["season","team","player_id","player_name","birth_date"]].copy()

# Span per (team, player) WHILE drafted-by-team primary (do NOT group on birth_date)
span = (prim.sort_values(["team","player_id","season"])
             .groupby(["team","player_id","player_name"], as_index=False)["season"]
             .agg(first_season="min", last_season="max"))

games_reg = games[games["game_type"].str.contains("REG")].copy()

def first_start_date(team, season, pid):
    mh = (games_reg["home_team"].eq(team) & games_reg["home_qb_id"].eq(pid) & games_reg["season"].eq(season))
    ma = (games_reg["away_team"].eq(team) & games_reg["away_qb_id"].eq(pid) & games_reg["season"].eq(season))
    d = games_reg.loc[mh | ma, "gameday"].min()
    return d if pd.notna(d) else season_anchor(season)

# First start date in first drafted season (vectorized via apply or loop)
firsts = []
for _, r in span.iterrows():
    firsts.append({
        "team": r["team"],
        "player_id": r["player_id"],
        "first_season": int(r["first_season"]),
        "first_start_date": first_start_date(r["team"], int(r["first_season"]), r["player_id"])
    })
firsts = pd.DataFrame(firsts)

# Next season with a DIFFERENT drafted primary on same team
team_year_primary = prim.set_index(["team","season"])["player_id"].to_dict()
def bench_season(team, last_season, pid):
    ny = int(last_season) + 1
    if (team, ny) in team_year_primary and team_year_primary[(team, ny)] != pid:
        return ny
    return np.nan
span["bench_season"] = [bench_season(r["team"], r["last_season"], r["player_id"]) for _, r in span.iterrows()]

def first_team_game_date(team, season):
    mask = (games_reg["season"].eq(season) & (games_reg["home_team"].eq(team) | games_reg["away_team"].eq(team)))
    d = games_reg.loc[mask, "gameday"].min()
    return d if pd.notna(d) else season_anchor(int(season))

benches = []
for _, r in span.dropna(subset=["bench_season"]).iterrows():
    benches.append({
        "team": r["team"],
        "player_id": r["player_id"],
        "bench_season": int(r["bench_season"]),
        "bench_date": first_team_game_date(r["team"], int(r["bench_season"]))
    })
benches = pd.DataFrame(benches) if benches else pd.DataFrame(columns=["team","player_id","bench_season","bench_date"])

# Build dev & merge birth_date once
dev = (span
       .merge(firsts, on=["team","player_id","first_season"], how="left")
       .merge(prim[["team","player_id","birth_date"]].drop_duplicates(), on=["team","player_id"], how="left"))
if not benches.empty:
    dev = dev.merge(benches, on=["team","player_id","bench_season"], how="left")
else:
    dev["bench_date"] = pd.NaT

# Ages (NaN if birth_date missing)
dev["first_start_age"] = [age_years(b, d) for b, d in zip(dev["birth_date"], dev["first_start_date"])]
dev["bench_age"]       = [age_years(b, d) for b, d in zip(dev["birth_date"], dev["bench_date"])]
dev["seasons_until_bench"] = dev["bench_season"] - dev["first_season"]

cohort = (dev.groupby("first_season", as_index=False)
            .agg(
                avg_first_start_age = ("first_start_age", "mean"),
                avg_bench_age       = ("bench_age", "mean"),
                avg_window_seasons  = ("seasons_until_bench", "mean"),
                n_qbs               = ("player_id", "nunique")
            )
            .rename(columns={"first_season":"season"}))

cohort.to_csv(OUT_DIR/"qb_dev_window_by_cohort_drafted_only.csv", index=False)
dev.to_csv(OUT_DIR/"qb_dev_window_rows_drafted_only.csv", index=False)

# =========================
# Sanity prints
# =========================
print("\n=== Drafted-by-team shares (2000–2024, season averages) ===")
print(f"Regular season (primary QBs): {overall_reg:.1f}%")
print(f"Playoffs (unique QBs)       : {overall_playoff:.1f}%")
print(f"Super Bowl QBs              : {overall_sb:.1f}%")

print("\nSaved:")
print("  rosters/yearly_pct_drafted_REG.csv")
print("  rosters/yearly_count_drafted_REG.csv")
print("  rosters/yearly_pct_drafted_playoffs.csv")
print("  rosters/playoff_qbs_drafted_only.csv")
print("  rosters/yearly_pct_drafted_superbowl.csv")
print("  rosters/superbowl_qbs_drafted_only.csv")
print("  rosters/qb_dev_window_by_cohort_drafted_only.csv")
print("  rosters/qb_dev_window_rows_drafted_only.csv")

print("\nDrafted-only starter counts (sample 1999–2003 window):")
print((reg_drafted.groupby('season')['team'].nunique()).reindex(range(1999,2004)))



=== Drafted-by-team shares (2000–2024, season averages) ===
Regular season (primary QBs): 55.2%
Playoffs (unique QBs)       : 52.3%
Super Bowl QBs              : 61.5%

Saved:
  rosters/yearly_pct_drafted_REG.csv
  rosters/yearly_count_drafted_REG.csv
  rosters/yearly_pct_drafted_playoffs.csv
  rosters/playoff_qbs_drafted_only.csv
  rosters/yearly_pct_drafted_superbowl.csv
  rosters/superbowl_qbs_drafted_only.csv
  rosters/qb_dev_window_by_cohort_drafted_only.csv
  rosters/qb_dev_window_rows_drafted_only.csv

Drafted-only starter counts (sample 1999–2003 window):
season
1999     NaN
2000    13.0
2001    11.0
2002    12.0
2003    13.0
Name: team, dtype: float64
